In [1]:
import json
import torch
import numpy as np
import pandas as pd



In [2]:
# Read all the data sets

set1_human = pd.read_json('dataset/set1_human.json')
set1_machine = pd.read_json('dataset/set1_machine.json')
set2_human = pd.read_json('dataset/set2_human.json')
set2_machine = pd.read_json('dataset/set2_machine.json')

# Add in label for all data sets
set1_human = set1_human.assign(label=1)
set2_human = set2_human.assign(label=1)
set1_machine = set1_machine.assign(label=0)
set2_machine = set2_machine.assign(label=0)

In [4]:
# Remove the machine id for testing
set1_machine = set1_machine.drop('machine_id', axis=1)
set2_machine = set2_machine.drop('machine_id', axis=1)

# Separate and concatenate into 2 sets, set 1 and set 2
set1 = pd.concat([set1_human, set1_machine])
set2 = pd.concat([set2_human, set2_machine])

# Split the labels and the data for training
x_set1 = pd.DataFrame(set1[['txt']])
y_set1 = pd.DataFrame(set1[['label']])
x_set2 = pd.DataFrame(set2[['txt']])
y_set2 = pd.DataFrame(set2[['label']])

In [10]:
x_set1

,txt
0,"[1479, 3457, 2277, 1827, 1493, 2582, 3723, 147..."
1,"[721, 2820, 1676, 4, 76, 2130, 1502, 1591, 148..."
2,"[2732, 3882, 1801, 1479, 2833, 2622, 2874, 156..."
3,"[1479, 4758, 1734, 1511, 1500, 1479, 91, 10, 8..."
4,"[4683, 1751, 2004, 1485, 1499, 15, 1574, 2193,..."
...,...
3495,"[76, 2044, 1703, 1484, 1479, 2705, 83, 2587, 8..."
3496,"[1518, 1502, 1585, 1686, 2996, 1568, 1533, 332..."
3497,"[1518, 76, 1752, 68, 2896, 1500, 2656, 15, 151..."
3498,"[76, 10, 80, 1478, 1479, 2926, 1500, 68, 3868,..."


In [11]:
x_set1['txt'] = x_set1['txt'].apply(lambda x: ' '.join(map(str, x)))

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer


vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(x_set1['txt'])
x_set1_vectorized = pd.DataFrame(vectorized_words.toarray())


In [18]:
print(x_set1_vectorized.shape)

(126084, 4981)


Split into training and testing sets

In [19]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

x_set1_train, x_set1_test, y_set1_train, y_set1_test= train_test_split(x_set1_vectorized, y_set1, test_size=0.2, stratify=y_set1)

In [20]:
x_set1_train

,0,1,2,3,4,5,6,7,8,9,...,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980
69177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51526,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60280,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125794,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77998,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76188,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63288,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14724,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73933,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

clf = LogisticRegression()
clf.fit(x_set1_train, y_set1_train)

y_pred = clf.predict(x_set1_test)
accuracy = accuracy_score(y_set1_test, y_pred)
print('Accuracy:', accuracy)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9796565808779791
